In [1]:
import numpy as np
import pandas as pd
#from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
#from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import mutual_info_regression
#from sklearn.linear_model import LassoCV
#from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_regression,SelectPercentile,SelectFpr
import seaborn as sns
from scipy import signal
#import pandas as pd

p = sns.color_palette()

#import pandas as pd
#import numpy as np
from sklearn.model_selection import cross_val_predict
#from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
#from sklearn.linear_model import ElasticNet
#from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
#from sklearn.feature_selection import f_classif
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Lasso
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

from sklearn.decomposition import PCA

from sklearn.feature_selection import SelectFromModel
#from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import matplotlib.pyplot as plt

import seaborn as sns


%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import mstats
from keras import optimizers
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
%matplotlib inline

Using Theano backend.


In [10]:
def neural_network(train_data, data, n=55413):
    newdata = pd.concat([train_data.ix[:,2:-2],train_data.ix[:,-1]], axis=1)
    for i in range(28):
        mad = newdata.ix[:,i].mad()
        dif = np.abs(newdata.ix[:,i] - np.median(newdata.ix[:,i]))
        newdata = newdata[dif <= 4*mad]
    data_filtered = train_data.ix[newdata.index]
    
    X = data_filtered.iloc[:,2:-2].values
    y = data_filtered.iloc[:,-1].values
    weight = data_filtered.iloc[:,-2].values

    #Extract optimal train data set
    X_train = X[-n:]
    y_train = y[-n:]
    weight_train = weight[-n:]
    X_test = data.iloc[:,2:-2].values
    y_test = data.iloc[:,-1].values
    
    weight_test = data.iloc[:,-2].values
    #lasso again
    X_scaler = StandardScaler(copy = True, with_mean = True, with_std = True)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.fit_transform(X_test)
    clf = LassoCV()
    sfm = SelectFromModel(clf)
    sfm.fit(X_train, y_train)
    X_train = sfm.transform(X_train)
    X_test = sfm.transform(X_test)
    def baseline_model():
        # create model
        model = Sequential()
        model.add(Dense(5, input_dim=len(X_train[0]), kernel_initializer='uniform', activation="tanh"))
        model.add(Dense(1, kernel_initializer='uniform'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasRegressor(build_fn=baseline_model,sample_weight = weight_train, epochs=150, batch_size=1500,
                                             verbose=0)))#callbacks=[change_lr] )))
    pipeline = Pipeline(estimators)
    pipeline.fit(X_train, y_train)#callbacks=callbacks_list)
    return pipeline.predict(X_test)

In [11]:
def regression_model(train_data, data, feature_num, cv_splits):
    # Scale data
    test_data = data

    train_x = train_data.drop(['y','weight','id'], axis = 1)
    train_y = train_data['y']
    train_sample_weight = train_data['weight']
    
    test_x = test_data.drop(['y','weight','id'], axis = 1)
    test_y = test_data['y']
    test_sample_weight = test_data['weight']

    scaler = StandardScaler().fit(train_x)
    scaled_train_x = scaler.transform(train_x)
    scaled_test_x = scaler.transform(test_x)
    
    # Select feature
    pca = PCA(n_components= feature_num).fit(scaled_train_x)
    selected_train_x = pca.transform(scaled_train_x)
    selected_test_x = pca.transform(scaled_test_x)
    
    # Model fit
    tscv = TimeSeriesSplit(n_splits = cv_splits)
    lasso_cv = LassoCV(cv = tscv)
    lasso_cv.fit(selected_train_x, train_y)
    
    test_y_predicted = lasso_cv.predict(selected_test_x)
    return test_y_predicted

In [12]:
def gradient_boost(params,X,X_holdout,y):
    '''gradient boosting
    Parameters
    ----------
    params: parameters in the gradient boosting
    X: data X
    y: data y
    k: split folds'''
    
    # scale data
    X_scaler = StandardScaler(copy = True, with_mean = True, with_std = True)
    X_train = X_scaler.fit_transform(X)
    X_test = X_scaler.transform(X_holdout)
    
    # lasso feature selection
    clf = LassoCV()
    sfm = SelectFromModel(clf)
    sfm.fit(X_train, y)
    X_train_lasso = sfm.transform(X_train)
    X_test_lasso = sfm.transform(X_test)
    
    # fit 
    est = GradientBoostingRegressor(**params)
    est.fit(X_train_lasso, y)
    y_predict = est.predict(X_test_lasso)
    
    return y_predict   

In [13]:
def final_model(train_data, data):
    y1 = regression_model(train_data, data, 18, 9)
    
    params = {'n_estimators':50, 'learning_rate':0.08, 'subsample':0.8,
          'max_depth': 1, 'random_state': 0,'min_samples_split': 5}
    
    
    X_holdout = data.drop(['y','timestamp','id'],1).values
    X = train_data.drop(['y','timestamp','id'],1).values
    y = train_data['y'].values
    y2 = gradient_boost(params,X,X_holdout,y)
    
    y3 =  neural_network(train_data, data, )
    
    ratio_1 = 0.1
    ratio_2 = 0.1
    ratio_3 = 0.8
    return ratio_1 * y1 + ratio_2 * y2 + ratio_3 * y3

In [14]:
def run_model(my_model, data):
    train_data = pd.read_pickle("C:/Users/JYJ/Downloads/train/ml_finalproj_train_vF.pkl")
    predicted_y = my_model(train_data, data)
    return r2_score(data['y'], predicted_y, data['weight'])

In [15]:
train_data = pd.read_pickle("ml_finalproj_train_vF.pkl")
data = pd.read_pickle("ml_finalproj_holdout.pkl")
predicted_y = final_model(train_data, data)

In [18]:
data['y'] = predicted_y

In [19]:
data['y']

0       -5.916806e-04
1        6.790082e-04
2       -2.194383e-04
3        3.876608e-04
4       -1.038230e-03
5       -1.538144e-04
6        4.912941e-04
7       -3.226350e-04
8       -7.921618e-04
9        3.126851e-04
10      -2.641546e-04
11      -1.000166e-03
12       1.665322e-04
13       8.126695e-05
14      -1.083846e-04
15      -4.781138e-04
16       1.384345e-04
17      -1.077992e-03
18      -2.908285e-04
19      -5.807687e-04
20       4.496707e-05
21      -1.699827e-04
22      -1.158355e-05
23      -8.026500e-04
24      -1.872352e-07
25      -1.920384e-04
26       7.530914e-05
27       4.750644e-04
28      -2.773804e-04
29      -1.492017e-04
             ...     
34683   -9.000669e-04
34684    2.160469e-04
34685    2.745437e-04
34686   -1.294714e-04
34687   -2.798820e-04
34688    2.924427e-04
34689   -8.387194e-04
34690    6.157406e-05
34691    1.481896e-04
34692   -3.007374e-04
34693   -3.893453e-04
34694    4.064684e-04
34695    8.051976e-04
34696   -8.111577e-04
34697    6

In [21]:
data.to_pickle("ml_finalproj_holdout_filled.pkl") 